# Генерация достоверных текстов

## Загрузка текстов

### Список книг
Задача: составить список текстов и сложить в dataframe

In [1]:
import os

import numpy as np
import pandas as pd

from src.model import BlockWithEmbedding, Block, Book

In [2]:
DATA_DIR = 'data/'

In [3]:
def get_row(root, file):
    parts = root.split('/')[1:]
    author = parts[0]
    series = '/'.join(parts[1:])
    title = file[:-4]
    path = os.path.join(root, file)
    return author, series, title, path


books = pd.DataFrame(
    [get_row(root, file) for root, dirs, files in os.walk(DATA_DIR) for file in files],
    columns=["author", "series", "title", "path"]
)

## Эмбеддинги
Задача: инициализировать модель

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cuda')
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cpu')

/home/yshmgrt/Programming/accurate-text-generation/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/yshmgrt/Programming/accurate-text-generation/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: No CUDA GPUs are available

In [5]:
embedding_model.similarity_fn_name

## База данных
Задача: создать базу данных с текстами и их эмбеддингами

In [6]:
from src.embedding_database import EmbeddingDatabase

In [7]:

with EmbeddingDatabase(sentence_transformer=embedding_model, file='main.db') as db:
    db.init_tables()
    book_list = books[:10].to_dict(orient='records')
    book_list = [Book(**book) for book in book_list]
    db.populate_db(book_list)

3.46.1
Populating database...


  0%|          | 0/10 [00:00<?, ?it/s]

[]


 10%|█         | 1/10 [00:00<00:05,  1.62it/s]

[]


...

 80%|████████  | 8/10 [00:03<00:00,  2.26it/s]

[]


 90%|█████████ | 9/10 [00:04<00:00,  2.52it/s]

[]


100%|██████████| 10/10 [00:04<00:00,  2.20it/s]


In [8]:
with EmbeddingDatabase(sentence_transformer=embedding_model, file='main.db') as db:
    print(db.get_similar_blocks(book_id=1, text='И так Гарри Поттер приобрел свою первую волшебную палочку'))

OperationalError: no such table: block

## Ранжирование блоков
Задача: отранжировать блоки по значимости с учетом их схожести с заданным текстом

In [7]:
from src.ranking import Ranker, TextRank

ranker: Ranker = TextRank(similarity=embedding_model.similarity)

## Все вместе


In [9]:
with EmbeddingDatabase(sentence_transformer=embedding_model) as db:
    bias = "И так Гарри Поттер приобрел свою первую волшебную палочку"
    book = db.get_book_by_title("Лавина")
    print(book)
    blocks = db.get_blocks_with_embedding_by_book(book[0].id)
    bias_embedding = embedding_model.encode(bias).astype(np.float32)
    print(ranker.rank(blocks, bias_block=BlockWithEmbedding(Block(0, 0, bias), bias_embedding)))

[Book(id=8, author='Нил Стивенсон', series='', title='Лавина', path='data/Нил Стивенсон/Лавина.txt'), Book(id=18, author='Нил Стивенсон', series='', title='Лавина', path='data/Нил Стивенсон/Лавина.txt')]


TypeError: new(): invalid data type 'bytes'

In [10]:
with EmbeddingDatabase(sentence_transformer=embedding_model, file='full.db') as db:
    bias = "После долгой битвы с Волан-де-Мортом Гарри попал в лазарет и..."
    book = db.get_book_by_title("1. Гарри Поттер и Филосовский Камень")
    blocks = db.get_similar_blocks(book[0].id, bias, n=10)
    print(blocks)

[Block(book_id=4021, position=139, text='- Дай сюда! - крикнул ему Гарри. - Или я собью тебя с метлы!\n- Да ну? - издевательски переспросил Малфой, однако, несмотря на тон, на лице его появилась озабоченность.\nГарри откуда-то знал, что ему надо делать. Он нагнулся вперед и крепко ухватился за метлу обеими руками, и она рванулась на Малфоя, как вылетевший из пращи камень. Малфой едва успел уклониться. А Гарри, проскочив мимо, резко развернулся и выровнял метлу. Снизу раздались аплодисменты.\n- Что, Малфой, заскучал? - громко крикнул Гарри. - Ты сейчас один, Крэбба и Гойла рядом нет, и никто тебе не поможет.\nКажется, Малфоя осенила та же мысль.\n- Тогда поймай, если сможешь! - заорал он и, метнув стеклянный шар высоко в небо, рванулся вниз, к земле.\nГарри видел, словно в замедленной съемке, как шар поднимается вверх, на мгновение застывает в воздухе, а потом начинает падать. Он нагнулся вперед и направил рукоятку метлы вниз, а в следующую секунду вошел в почти отвесное пике. Скорость 

## Сбор обучающего датасета

## RAG - первая попытка

### Сбор контекста

In [1]:
def get_context(
        file="data/Джоан Кэтлин Роулинг/FB2/Гарри_Поттер/Перевод_РОСМЭН/1. Гарри Поттер и Филосовский Камень.txt",
        block_size=1000):
    blocks = Splitter(block_max_length=block_size).split(read_text(file))
    embeddings = get_embedding(blocks)

    def inner(line, length=10):
        emb = get_embedding([line])
        return get_n_best(blocks, get_similarity(embeddings, emb), length)

    return inner

In [18]:
context_retriever = get_context()

In [19]:
print('\n\n-------\n\n'.join(map(lambda x: x[1], context_retriever(
    "И все прошло как по маслу: Тролль был повержен, Пушок - спасен, Лестницы повернулись, куда надо"))))

- Продолжай играть! - шепнул Рон, когда они сняли с себя мантию и медленно двинулись к люку, который охранял Пушок. Жаркое зловонное дыхание, вырывавшееся из трех пастей, чувствовалось все сильнее. - Думаю, мы легко откроем люк, - заверил их Рон, вставая на цыпочки и бросая взгляд за спину Пушка. - Хочешь пойти первой, Гермиона?
- Нет, ни за что! - воскликнула та, отступая назад.
- Хорошо. - Рон скрипнул зубами, собираясь с силами, и опасливо переступил через лапы Пушка. А потом нагнулся над люком и потянул за кольцо.
- Что ты там видишь? - возбужденно прошептала Гермиона.
- Ничего. Темнота. Никаких ступеней не видно, придется прыгать.
Гарри, продолжавший играть на флейте, поднял руку и помахал, привлекая внимание Рона. А потом указал пальцем на себя.
- Ты хочешь пойти первым? Уверен? - переспросил Рон. - Честно говоря, не знаю, как далеко нам придется лететь. Отдай флейту Гермионе, Пушок не должен проснуться.

-------

- Ты в порядке, Гермиона? - прошептал Хагрид. - Не волнуйся, найде

In [20]:
def make_prompt(query, with_context=True):
    context = ''
    if with_context:
        context = '\n\n-------\n\n'.join(map(lambda x: x[1], context_retriever(query)))
    return """
        {}
        {}
    """.format(context, query)

In [21]:
prompt = make_prompt("И все прошло как по маслу: Тролль был повержен, Пушок - спасен, Лестницы повернулись, куда надо")


In [1]:
from unsloth import FastLanguageModel

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",  # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",  # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",  # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",  # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",  # Gemma 2x faster!
]  # More models at https://huggingface.co/unsloth

embedding_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Laptop GPU. Max memory: 7.653 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(embedding_model)  # Enable native 2x faster prompt


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [ ]:
prompt = "Hello, how are you?"
tokens = tokenizer([prompt], return_tensors="pt").to("cuda")
embedding_model.generate(**tokens)


In [28]:
def generate_text(prompt):
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = embedding_model.generate(**inputs, max_new_tokens=512, use_cache=True)
    return tokenizer.batch_decode(outputs)


In [29]:
generate_text(
    prompt
)

/home/yshmgrt/Programming/the-dialogue-system/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1498: SyntaxWarning: invalid escape sequence '\ '
  f"   \\\   /|    GPU: {gpu_stats.name}. Max memory: {max_memory} GB. Platform = {platform_system}.\n"\
/home/yshmgrt/Programming/the-dialogue-system/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1499: SyntaxWarning: invalid escape sequence '\_'
  f"O^O/ \_/ \\    Pytorch: {torch.__version__}. CUDA = {gpu_stats.major}.{gpu_stats.minor}. CUDA Toolkit = {torch.version.cuda}.\n"\
/home/yshmgrt/Programming/the-dialogue-system/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1500: SyntaxWarning: invalid escape sequence '\ '
  f"\        /    Bfloat16 = {str(SUPPORTS_BFLOAT16).upper()}. FA [Xformers = {xformers_version}. FA2 = {HAS_FLASH_ATTENTION}]\n"\
/home/yshmgrt/Programming/the-dialogue-system/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1647: SyntaxWarning: invalid escape sequence '\.'
  start = 

ValueError: The following `model_kwargs` are not used by the model: ['load_in_4bit'] (note: typos in the generate arguments will also show up in this list)

In [27]:
generate_text(
    make_prompt("И все прошло как по маслу: Тролль был повержен, Пушок - спасен, Лестницы повернулись, куда надо",
                with_context=False)
)

['<|begin_of_text|>\n        \n        И все прошло как по маслу: Тролль был повержен, Пушок - спасен, Лестницы повернулись, куда надо\n    <|end_of_text|>']